In [1]:
import torch
from flcore.pflniid_utils.data_utils import read_client_data
from flcore.clients.clientavg import clientAVG

In [2]:
import copy
import argparse
import os
import time
import warnings
import numpy as np
import logging
from flcore.servers.serveravg import FedAvg
from flcore.servers.serverlocal import Local

In [3]:
parser = argparse.ArgumentParser()

In [4]:
# general
parser.add_argument('-go', "--goal", type=str, default="test", 
                    help="The goal for this experiment")
parser.add_argument('-dev', "--device", type=str, default="cpu",  # KAI: Changed the default to cpu
                    choices=["cpu", "cuda"])
parser.add_argument('-did', "--device_id", type=str, default="0")
parser.add_argument('-data', "--dataset", type=str, default="cphs")  # KAI: Changed the default to cphs (from mnist)
#parser.add_argument('-nb', "--num_classes", type=int, default=10)  # Not doing classification...
parser.add_argument('-m', "--model", type=str, default="Linear Regression")  # KAI: Changed the default to Linear Regression
parser.add_argument('-lbs', "--batch_size", type=int, default=1200)  # Setting it to a full update would be 1300ish... how many batches does it run? In one epoch? Not even sure where that is set
# The 1300 and the batch size are 2 separate things...
# I want to restrict the given dataset to just the 1300, but then iterate in batches... or do I since we don't have that much data and can probably just use all the data at once? Make batch size match the update size? ...
parser.add_argument('-lr', "--local_learning_rate", type=float, default=0.005,
                    help="Local learning rate")
parser.add_argument('-ld', "--learning_rate_decay", type=bool, default=False)
parser.add_argument('-ldg', "--learning_rate_decay_gamma", type=float, default=0.99)
parser.add_argument('-gr', "--global_rounds", type=int, default=250)  # KAI: Switched to 250 down from 2000
parser.add_argument('-ls', "--local_epochs", type=int, default=1, 
                    help="Multiple update steps in one local epoch.")  # KAI: I think it was 1 originally.  I'm gonna keep it there.  Does this mean I can set batchsize to 1300 and cook?Is my setup capable or running multiple epochs? Implicitly I was doing 1 epoch before, using the full update data I believe...
parser.add_argument('-algo', "--algorithm", type=str, default="FedAvg")
parser.add_argument('-jr', "--join_ratio", type=float, default=0.2,
                    help="Ratio of clients per round")
parser.add_argument('-rjr', "--random_join_ratio", type=bool, default=False,
                    help="Random ratio of clients per round")
parser.add_argument('-nc', "--num_clients", type=int, default=14,
                    help="Total number of clients")
parser.add_argument('-dp', "--privacy", type=bool, default=False,
                    help="differential privacy")
parser.add_argument('-dps', "--dp_sigma", type=float, default=0.0)
parser.add_argument('-sfn', "--save_folder_name", type=str, default='items')

# SECTION: practical
parser.add_argument('-cdr', "--client_drop_rate", type=float, default=0.0,
                    help="Rate for clients that train but drop out")
parser.add_argument('-tsr', "--train_slow_rate", type=float, default=0.0,
                    help="The rate for slow clients when training locally")
parser.add_argument('-ssr', "--send_slow_rate", type=float, default=0.0,
                    help="The rate for slow clients when sending global model")
parser.add_argument('-ts', "--time_select", type=bool, default=False,
                    help="Whether to group and select clients at each round according to time cost")
parser.add_argument('-tth', "--time_threthold", type=float, default=10000,
                    help="The threthold for droping slow clients")

# SECTION: Kai's additional args
parser.add_argument('-pca_channels', "--pca_channels", type=int, default=64,
                    help="Number of principal components. 64 means do not use any PCA")
parser.add_argument('-lambdas', "--lambdas", type=list, default=[0, 1e-3, 1e-4],
                    help="Lamda F, D, E penalty terms ")
parser.add_argument('-starting_update', "--starting_update", type=int, default=0,
                    help="Which update to start on (for CPHS Simulation). Use 0 or 10.")
parser.add_argument('-test_split', "--test_split", type=float, default=0.2,
                    help="Percent of data to use for testing")
parser.add_argument('-device_channels', "--device_channels", type=int, default=64,
                    help="Number of recording channels with the used EMG device")
parser.add_argument('-dt', "--dt", type=float, default=1/60,
                    help="Delta time, amount of time (sec?) between measurements")
parser.add_argument('-normalize_emg', "--normalize_emg", type=bool, default=False,
                    help="Normalize the input EMG signals")
parser.add_argument('-normalize_V', "--normalize_V", type=bool, default=False,
                    help="Normalize the V term in the cost function")
parser.add_argument('-local_round_threshold', "--local_round_threshold", type=int, default=50,
                    help="Number of communication rounds per client until a client will advance to the next batch of streamed data")
parser.add_argument('-debug_mode', "--debug_mode", type=bool, default=False,
                    help="In debug mode, the code is run to minimize overhead time in order to debug as fast as possible.  Namely, the data is held at the server to decrease init time, and communication delays are ignored.")
parser.add_argument('-condition_number', "--condition_number", type=int, default=1,
                    help="Which condition number (trial) to train on")



parser.add_argument('-t', "--times", type=int, default=1,
                    help="Running times")
parser.add_argument('-ab', "--auto_break", type=bool, default=False)
parser.add_argument('-dlg', "--dlg_eval", type=bool, default=False)
parser.add_argument('-eg', "--eval_gap", type=int, default=1,
                    help="Rounds gap for evaluation")
parser.add_argument('-nnc', "--num_new_clients", type=int, default=0)


# This one for sure breaks it
#parser.add_argument('-fte', "--fine_tuning_epoch", type=int, default=0)

_StoreAction(option_strings=['-nnc', '--num_new_clients'], dest='num_new_clients', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [5]:
# SECTION: Idk what these are lol
'''
parser.add_argument('-eg', "--eval_gap", type=int, default=1,
                    help="Rounds gap for evaluation")
parser.add_argument('-pv', "--prev", type=int, default=0,
                    help="Previous Running times")
parser.add_argument('-t', "--times", type=int, default=1,
                    help="Running times")
parser.add_argument('-ab', "--auto_break", type=bool, default=False)
parser.add_argument('-dlg', "--dlg_eval", type=bool, default=False)  # DLG = Deep Leakage from Gradients
parser.add_argument('-dlgg', "--dlg_gap", type=int, default=100)
parser.add_argument('-bnpc', "--batch_num_per_client", type=int, default=2)  # Only used with DLG
parser.add_argument('-nnc', "--num_new_clients", type=int, default=0)
parser.add_argument('-fte', "--fine_tuning_epoch", type=int, default=0)
'''
#args = parser.parse_known_args()
0

0

In [6]:
#args = parser.parse_args()
args = parser.parse_known_args()

In [7]:
args

(Namespace(goal='test', device='cpu', device_id='0', dataset='cphs', model='LinearRegression', batch_size=1200, local_learning_rate=0.005, learning_rate_decay=False, learning_rate_decay_gamma=0.99, global_rounds=250, local_epochs=1, algorithm='FedAvg', join_ratio=0.2, random_join_ratio=False, num_clients=14, privacy=False, dp_sigma=0.0, save_folder_name='items', client_drop_rate=0.0, train_slow_rate=0.0, send_slow_rate=0.0, time_select=False, time_threthold=10000, pca_channels=64, lambdas=[0, 0.001, 0.0001], starting_update=0, test_split=0.2, device_channels=64, dt=0.016666666666666666, normalize_emg=False, normalize_V=False, local_round_threshold=50, debug_mode=False, condition_number=1, times=1, auto_break=False, dlg_eval=False, eval_gap=1, num_new_clients=0),
 ['-f',
  'C:\\Users\\kdmen\\AppData\\Roaming\\jupyter\\runtime\\kernel-fcf87f18-9cd8-43af-a01e-ddbeca581a9f.json'])

In [8]:
args = args[0]
args

Namespace(goal='test', device='cpu', device_id='0', dataset='cphs', model='LinearRegression', batch_size=1200, local_learning_rate=0.005, learning_rate_decay=False, learning_rate_decay_gamma=0.99, global_rounds=250, local_epochs=1, algorithm='FedAvg', join_ratio=0.2, random_join_ratio=False, num_clients=14, privacy=False, dp_sigma=0.0, save_folder_name='items', client_drop_rate=0.0, train_slow_rate=0.0, send_slow_rate=0.0, time_select=False, time_threthold=10000, pca_channels=64, lambdas=[0, 0.001, 0.0001], starting_update=0, test_split=0.2, device_channels=64, dt=0.016666666666666666, normalize_emg=False, normalize_V=False, local_round_threshold=50, debug_mode=False, condition_number=1, times=1, auto_break=False, dlg_eval=False, eval_gap=1, num_new_clients=0)

In [9]:
#args = parser.parse_args()

# I always need to run on CPU only since I don't have Nvidia GPU available
#os.environ["CUDA_VISIBLE_DEVICES"] = args.device_id
#if args.device == "cuda" and not torch.cuda.is_available():
#    print("\ncuda is not avaiable.\n")
#    args.device = "cpu"

#print("=" * 50)

print("Algorithm: {}".format(args.algorithm))
print("Local batch size: {}".format(args.batch_size))
print("Local steps: {}".format(args.local_epochs))
print("Local learing rate: {}".format(args.local_learning_rate))
print("Local learing rate decay: {}".format(args.learning_rate_decay))
if args.learning_rate_decay:
    print("Local learing rate decay gamma: {}".format(args.learning_rate_decay_gamma))
print("Total number of clients: {}".format(args.num_clients))
print("Clients join in each round: {}".format(args.join_ratio))
print("Clients randomly join: {}".format(args.random_join_ratio))
print("Client drop rate: {}".format(args.client_drop_rate))
print("Client select regarding time: {}".format(args.time_select))
if args.time_select:
    print("Time threthold: {}".format(args.time_threthold))
print("Running times: {}".format(args.times))
print("Dataset: {}".format(args.dataset))
print("Backbone: {}".format(args.model))
print("Using device: {}".format(args.device))
print("Using DP: {}".format(args.privacy))
if args.privacy:
    print("Sigma for DP: {}".format(args.dp_sigma))
print("Auto break: {}".format(args.auto_break))
if not args.auto_break:
    print("Global rounds: {}".format(args.global_rounds))
if args.device == "cuda":
    print("Cuda device id: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))
print("DLG attack: {}".format(args.dlg_eval))
if args.dlg_eval:
    print("DLG attack round gap: {}".format(args.dlg_gap))
print("Total number of new clients: {}".format(args.num_new_clients))
#print("Fine tuning epoches on new clients: {}".format(args.fine_tuning_epoch))

print("KAI'S ADDITIONS")
if args.pca_channels!=64:
    print("Number of PCA Components Used: {}".format(args.pca_channels))
print("Lambda penalty terms (F, D, E): {}".format(args.lambdas))
print("Starting update: {}".format(args.starting_update))
print("Testing split: {}".format(args.test_split))
if args.dt!=1/60:
    print("dt: {}".format(args.dt))
print("Normalize EMG input: {}".format(args.normalize_emg))
print("Normalize V term: {}".format(args.normalize_V))
print("Local round threshold: {}".format(args.local_round_threshold))
print("In Debug Mode: {}".format(args.debug_mode))

print("=" * 50)

Algorithm: FedAvg
Local batch size: 1200
Local steps: 1
Local learing rate: 0.005
Local learing rate decay: False
Total number of clients: 14
Clients join in each round: 0.2
Clients randomly join: False
Client drop rate: 0.0
Client select regarding time: False
Running times: 1
Dataset: cphs
Backbone: LinearRegression
Using device: cpu
Using DP: False
Auto break: False
Global rounds: 250
DLG attack: False
Total number of new clients: 0
KAI'S ADDITIONS
Lambda penalty terms (F, D, E): [0, 0.001, 0.0001]
Starting update: 0
Testing split: 0.2
Normalize EMG input: False
Normalize V term: False
Local round threshold: 50
In Debug Mode: False


In [10]:
dataset = 'cphs'

In [11]:
# All the selfs aren't defined since this is outside the class

def _train(self):
    trainloader = self.load_train_data()
    self.model.train()

    start_time = time.time()

    max_local_steps = self.local_epochs

    for step in range(max_local_steps):
        for i, (x, y) in enumerate(trainloader):
            print(f"Step {step}, pair {i} in traindl")
            print(f"x.size(): {x.size()}")
            if type(x) == type([]):
                x[0] = x[0].to(self.device)
            else:
                x = x.to(self.device)
            y = y.to(self.device)
            output = self.model(x)
            loss = self.loss(output, y, self.model)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

In [12]:
my_clients = []
for i in range(14):
    train_data = read_client_data(dataset, i, is_train=True)
    test_data = read_client_data(dataset, i, is_train=False)
    client = clientAVG(args, 
                    ID=i, 
                    train_samples=len(train_data), 
                    test_samples=len(test_data), 
                    train_slow=False, 
                    send_slow=False)
    my_clients.append(client)

Client0, train=True: read_data() called!
Client0, train=False: read_data() called!


AttributeError: 'str' object has no attribute 'children'

In [ ]:
my_clients.train()